# 개선된 다중 에이전트 백테스팅 시스템 (데이터 품질 검증 포함)

5개 투자 에이전트의 성과를 백테스팅하되, 데이터 품질 문제(중복 티커, 오타 등)를 자동으로 감지하고 수정합니다.

## 🔧 개선 사항
- ✅ **중복 티커 감지 및 가중치 합산**
- ✅ **티커 오타 자동 수정** (예: CPTR → CPRT)
- ✅ **유효하지 않은 티커 제거**
- ✅ **가중치 정규화** (합계가 100%가 되도록)
- ✅ **데이터 품질 리포트 제공**


In [ ]:
# 개선된 다중 에이전트 백테스팅 시스템 (1/3) - 데이터 품질 검증 클래스
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from collections import Counter
import difflib
warnings.filterwarnings('ignore')

import yfinance as yf

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

class ImprovedMultiAgentBacktester:
    """데이터 품질 검증이 포함된 5개 투자 에이전트 통합 백테스팅 클래스"""
    
    def __init__(self, 
                 results_dir="results",
                 ohlcv_path="data/nasdaq100_ohlcv.csv",
                 benchmark_cache_path="data/benchmark_data.csv",
                 transaction_cost=0.0001):
        
        self.results_dir = results_dir
        self.ohlcv_path = ohlcv_path
        self.benchmark_cache_path = benchmark_cache_path
        self.transaction_cost = transaction_cost
        
        # 에이전트 정보 정의
        self.agents = {
            'graham': {
                'name': 'Benjamin Graham',
                'directory': 'graham_agent',
                'portfolio_prefix': 'graham_portfolio_',
                'color': 'blue'
            },
            'buffett': {
                'name': 'Warren Buffett',
                'directory': 'buffett_agent',
                'portfolio_prefix': 'buffett_portfolio_',
                'color': 'red'
            },
            'greenblatt': {
                'name': 'Joel Greenblatt',
                'directory': 'greenblatt_agent',
                'portfolio_prefix': 'greenblatt_portfolio_',
                'color': 'green'
            },
            'piotroski': {
                'name': 'Joseph Piotroski',
                'directory': 'piotroski_agent',
                'portfolio_prefix': 'piotroski_portfolio_',
                'color': 'orange'
            },
            'altman': {
                'name': 'Edward Altman',
                'directory': 'altman_agent',
                'portfolio_prefix': 'altman_portfolio_',
                'color': 'purple'
            }
        }
        
        # 벤치마크 티커들 정의
        self.benchmarks = {
            'QQQ': {'name': 'NASDAQ 100 ETF', 'color': 'black'},
            'SPY': {'name': 'S&P 500 ETF', 'color': 'gray'}
        }
        
        # 백테스팅 결과 저장용
        self.backtest_results = {}
        
        # 데이터 품질 문제 로그
        self.data_quality_issues = []
        
    def validate_and_clean_ticker(self, ticker, valid_tickers=None):
        """티커 검증 및 정정"""
        if valid_tickers is None:
            # NASDAQ 100 티커 목록에서 유효한 티커 확인
            valid_tickers = set(self.ohlcv_df['TICKERSYMBOL'].unique())
        
        ticker = ticker.strip().upper()
        
        # 정확한 매치가 있으면 그대로 반환
        if ticker in valid_tickers:
            return ticker
        
        # 유사한 티커 찾기 (오타 수정)
        close_matches = difflib.get_close_matches(ticker, valid_tickers, n=1, cutoff=0.8)
        if close_matches:
            corrected_ticker = close_matches[0]
            self.data_quality_issues.append({
                'type': 'ticker_correction',
                'original': ticker,
                'corrected': corrected_ticker,
                'message': f"티커 '{ticker}'를 '{corrected_ticker}'로 수정했습니다."
            })
            return corrected_ticker
        
        # 매치되는 것이 없으면 None 반환
        self.data_quality_issues.append({
            'type': 'invalid_ticker',
            'original': ticker,
            'message': f"유효하지 않은 티커 '{ticker}'가 발견되었습니다."
        })
        return None


In [ ]:
# 데이터 정리 및 검증 메서드들 (2/3)

def clean_portfolio_data(self, df, agent_key):
    """포트폴리오 데이터 정리 및 검증"""
    if df is None or len(df) == 0:
        return df
        
    # 원본 데이터 백업
    original_df = df.copy()
    
    # 1. 빈 행 제거
    df = df.dropna(subset=['Ticker'])
    
    # 2. 티커 검증 및 정정
    valid_tickers = set(self.ohlcv_df['TICKERSYMBOL'].unique())
    df['Original_Ticker'] = df['Ticker'].copy()
    df['Ticker'] = df['Ticker'].apply(lambda x: self.validate_and_clean_ticker(x, valid_tickers))
    
    # 3. 유효하지 않은 티커 제거
    invalid_mask = df['Ticker'].isnull()
    if invalid_mask.any():
        invalid_tickers = df[invalid_mask]['Original_Ticker'].tolist()
        self.data_quality_issues.append({
            'type': 'removed_invalid_tickers',
            'agent': self.agents[agent_key]['name'],
            'tickers': invalid_tickers,
            'message': f"{self.agents[agent_key]['name']}에서 유효하지 않은 티커 {len(invalid_tickers)}개를 제거했습니다: {invalid_tickers}"
        })
        df = df[~invalid_mask]
    
    # 4. 중복 티커 처리
    duplicate_tickers = df[df.duplicated('Ticker', keep=False)]['Ticker'].tolist()
    if duplicate_tickers:
        # 중복된 티커들의 가중치를 합산
        self.data_quality_issues.append({
            'type': 'duplicate_tickers',
            'agent': self.agents[agent_key]['name'],
            'tickers': list(set(duplicate_tickers)),
            'message': f"{self.agents[agent_key]['name']}에서 중복 티커 {len(set(duplicate_tickers))}개를 발견하여 가중치를 합산했습니다: {list(set(duplicate_tickers))}"
        })
        
        # 중복 티커의 가중치 합산
        df = df.groupby('Ticker').agg({
            'Weight (%)': 'sum',
            'Score': 'mean',  # Score는 평균
            'Reason': 'first'  # Reason은 첫 번째 것 사용
        }).reset_index()
    
    # 5. 가중치 정규화
    total_weight = df['Weight (%)'].sum()
    if abs(total_weight - 100.0) > 0.01:  # 0.01% 이상 차이나면 정규화
        self.data_quality_issues.append({
            'type': 'weight_normalization',
            'agent': self.agents[agent_key]['name'],
            'original_total': total_weight,
            'message': f"{self.agents[agent_key]['name']}의 총 가중치가 {total_weight:.2f}%였으므로 100%로 정규화했습니다."
        })
        df['Weight (%)'] = df['Weight (%)'] / total_weight * 100.0
    
    # 6. 가중치가 0 이하인 항목 제거
    zero_weight_mask = df['Weight (%)'] <= 0
    if zero_weight_mask.any():
        zero_weight_tickers = df[zero_weight_mask]['Ticker'].tolist()
        self.data_quality_issues.append({
            'type': 'removed_zero_weight',
            'agent': self.agents[agent_key]['name'],
            'tickers': zero_weight_tickers,
            'message': f"{self.agents[agent_key]['name']}에서 가중치가 0 이하인 티커 {len(zero_weight_tickers)}개를 제거했습니다: {zero_weight_tickers}"
        })
        df = df[~zero_weight_mask]
    
    # Original_Ticker 컬럼 제거
    if 'Original_Ticker' in df.columns:
        df = df.drop('Original_Ticker', axis=1)
    
    return df
    
def print_data_quality_report(self):
    """데이터 품질 문제 리포트 출력"""
    if not self.data_quality_issues:
        print("✅ 데이터 품질 검사 완료: 문제가 발견되지 않았습니다.")
        return
        
    print("\\n" + "=" * 80)
    print("📋 데이터 품질 문제 리포트")
    print("=" * 80)
    
    # 문제 유형별 분류
    issue_types = Counter([issue['type'] for issue in self.data_quality_issues])
    
    print(f"\\n총 {len(self.data_quality_issues)}개의 데이터 품질 문제를 발견하고 수정했습니다:")
    for issue_type, count in issue_types.items():
        type_names = {
            'ticker_correction': '티커 오타 수정',
            'invalid_ticker': '유효하지 않은 티커',
            'duplicate_tickers': '중복 티커',
            'weight_normalization': '가중치 정규화',
            'removed_invalid_tickers': '유효하지 않은 티커 제거',
            'removed_zero_weight': '0 가중치 티커 제거'
        }
        print(f"  - {type_names.get(issue_type, issue_type)}: {count}건")
    
    print("\\n[상세 내용]")
    for i, issue in enumerate(self.data_quality_issues, 1):
        print(f"{i:2d}. {issue['message']}")
    
    print("\\n" + "=" * 80)

# 클래스에 메서드 추가
ImprovedMultiAgentBacktester.clean_portfolio_data = clean_portfolio_data
ImprovedMultiAgentBacktester.print_data_quality_report = print_data_quality_report


In [ ]:
# 기존 MultiAgentBacktester의 나머지 메서드들을 복사해서 사용
# (벤치마크 데이터, 백테스팅 계산, 시각화 등)

# 기존 04_multi_agent_backtesting.ipynb의 메서드들을 그대로 사용하되 
# 데이터 로딩 부분에 품질 검증을 추가

def load_data(self):
    """필요한 데이터들을 로드 (품질 검증 포함)"""
    print("데이터 로딩 중...")
    
    # OHLCV 데이터 로드
    print("  - OHLCV 데이터 로딩...")
    self.ohlcv_df = pd.read_csv(self.ohlcv_path)
    self.ohlcv_df['EVAL_D'] = pd.to_datetime(self.ohlcv_df['EVAL_D'])
    
    # 각 에이전트별 포트폴리오 파일들 로드
    print("  - 에이전트별 포트폴리오 결과 파일들 로딩...")
    self.agent_portfolios = {}
    for agent_key, agent_info in self.agents.items():
        self.load_agent_portfolio_files(agent_key, agent_info)
    
    # 벤치마크 데이터 로드
    print("  - 벤치마크 데이터 로딩...")
    self.benchmark_df = self.load_benchmark_data()
    
    print("데이터 로딩 완료!")
    
    # 데이터 품질 리포트 출력
    self.print_data_quality_report()
    
def load_agent_portfolio_files(self, agent_key, agent_info):
    """특정 에이전트의 포트폴리오 CSV 파일들을 로드 (데이터 품질 검증 포함)"""
    portfolio_dir = os.path.join(self.results_dir, agent_info['directory'])
    portfolio_files = glob.glob(f"{portfolio_dir}/{agent_info['portfolio_prefix']}*.csv")
    
    portfolios = []
    
    for file in sorted(portfolio_files):
        try:
            # 파일명에서 날짜 추출
            filename = os.path.basename(file)
            date_part = filename.replace(agent_info['portfolio_prefix'], '').replace('.csv', '')
            start_date, end_date = date_part.split('_')
            
            # CSV 파일 로드
            df = pd.read_csv(file)
            if len(df) == 0 or (len(df) == 1 and df.iloc[0].isnull().all()):
                continue
            
            # 🔧 데이터 품질 검증 및 정리 (이 부분이 핵심 개선사항!)
            df = self.clean_portfolio_data(df, agent_key)
            
            if len(df) > 0:
                df['analysis_start'] = pd.to_datetime(start_date)
                df['analysis_end'] = pd.to_datetime(end_date)
                df['investment_start'] = df['analysis_end'] + timedelta(days=1)
                next_quarter_start = df['analysis_end'] + timedelta(days=90)
                df['investment_end'] = next_quarter_start
                
                portfolios.append(df)
                
        except Exception as e:
            print(f"파일 로딩 실패: {file}, 오류: {e}")
            
    if portfolios:
        self.agent_portfolios[agent_key] = pd.concat(portfolios, ignore_index=True)
        print(f"  - {agent_info['name']}: {len(portfolios)}개 분기 포트폴리오 로드됨")
    else:
        print(f"  - {agent_info['name']}: 포트폴리오 데이터 없음")
        self.agent_portfolios[agent_key] = None

# 클래스에 메서드 추가
ImprovedMultiAgentBacktester.load_data = load_data
ImprovedMultiAgentBacktester.load_agent_portfolio_files = load_agent_portfolio_files


In [ ]:
# 개선된 백테스터 초기화 및 데이터 품질 검사 실행
print("🔧 개선된 다중 에이전트 백테스터 초기화 중...")
backtester = ImprovedMultiAgentBacktester(transaction_cost=0.0001)

# 데이터 로딩 (자동으로 품질 검증 수행)
backtester.load_data()


In [ ]:
# Edward Altman 에이전트의 데이터 품질 문제 직접 확인
import pandas as pd
import os
import glob

def check_altman_data_quality():
    """Altman 에이전트의 데이터 품질 문제를 확인"""
    print("🔍 Edward Altman 에이전트 데이터 품질 검사")
    print("=" * 60)
    
    portfolio_dir = "results/altman_agent"
    portfolio_files = glob.glob(f"{portfolio_dir}/altman_portfolio_*.csv")
    
    issues_found = []
    
    for file in sorted(portfolio_files):
        filename = os.path.basename(file)
        print(f"\\n📁 {filename}")
        
        try:
            df = pd.read_csv(file)
            if len(df) == 0:
                continue
                
            # 1. 중복 티커 검사
            duplicates = df[df.duplicated('Ticker', keep=False)]
            if len(duplicates) > 0:
                duplicate_tickers = duplicates['Ticker'].unique().tolist()
                print(f"  ⚠️  중복 티커 발견: {duplicate_tickers}")
                issues_found.append(f"{filename}: 중복 티커 {duplicate_tickers}")
                
                # 중복 상세 정보 출력
                for ticker in duplicate_tickers:
                    ticker_rows = df[df['Ticker'] == ticker]
                    print(f"    - {ticker}: {len(ticker_rows)}번 등장, 가중치 합계: {ticker_rows['Weight (%)'].sum():.2f}%")
            
            # 2. 가중치 합계 검사
            total_weight = df['Weight (%)'].sum()
            if abs(total_weight - 100.0) > 0.01:
                print(f"  ⚠️  가중치 합계 이상: {total_weight:.2f}% (100%가 아님)")
                issues_found.append(f"{filename}: 가중치 합계 {total_weight:.2f}%")
            
            # 3. 이상한 티커 검사 (예: CPTR)
            suspicious_tickers = []
            for ticker in df['Ticker'].unique():
                if ticker in ['CPTR']:  # CPRT의 오타로 추정
                    suspicious_tickers.append(ticker)
            
            if suspicious_tickers:
                print(f"  ⚠️  의심스러운 티커: {suspicious_tickers} (오타 가능성)")
                issues_found.append(f"{filename}: 의심스러운 티커 {suspicious_tickers}")
            
            if len(duplicates) == 0 and abs(total_weight - 100.0) <= 0.01 and len(suspicious_tickers) == 0:
                print("  ✅ 문제 없음")
                
        except Exception as e:
            print(f"  ❌ 파일 읽기 오류: {e}")
            issues_found.append(f"{filename}: 파일 읽기 오류")
    
    print(f"\\n📋 총 {len(issues_found)}개의 데이터 품질 문제 발견:")
    for issue in issues_found:
        print(f"  - {issue}")
    
    return issues_found

# 데이터 품질 검사 실행
issues = check_altman_data_quality()


In [ ]:
# 데이터 품질 검사 실행
issues = check_altman_data_quality()


In [ ]:
# 데이터 품질 문제 자동 수정 함수
import shutil

def fix_altman_data_quality():
    """Altman 에이전트의 데이터 품질 문제를 자동으로 수정"""
    print("🔧 Edward Altman 에이전트 데이터 품질 자동 수정")
    print("=" * 60)
    
    portfolio_dir = "results/altman_agent"
    portfolio_files = glob.glob(f"{portfolio_dir}/altman_portfolio_*.csv")
    
    fixes_applied = []
    
    for file in sorted(portfolio_files):
        filename = os.path.basename(file)
        print(f"\\n📁 {filename} 처리 중...")
        
        try:
            df = pd.read_csv(file)
            if len(df) == 0:
                continue
                
            original_df = df.copy()
            fixes_for_file = []
            
            # 1. 티커 오타 수정 (CPTR → CPRT)
            if 'CPTR' in df['Ticker'].values:
                df.loc[df['Ticker'] == 'CPTR', 'Ticker'] = 'CPRT'
                fixes_for_file.append("CPTR → CPRT 오타 수정")
                print("  ✅ CPTR를 CPRT로 수정했습니다.")
            
            # 2. 중복 티커 처리 (가중치 합산)
            duplicates = df[df.duplicated('Ticker', keep=False)]
            if len(duplicates) > 0:
                duplicate_tickers = duplicates['Ticker'].unique().tolist()
                print(f"  🔄 중복 티커 처리: {duplicate_tickers}")
                
                # 중복 티커의 가중치 합산
                df_fixed = df.groupby('Ticker').agg({
                    'Weight (%)': 'sum',
                    'Score': 'mean',
                    'Reason': 'first'
                }).reset_index()
                
                df = df_fixed
                fixes_for_file.append(f"중복 티커 {len(duplicate_tickers)}개 가중치 합산: {duplicate_tickers}")
                
                for ticker in duplicate_tickers:
                    original_weight = original_df[original_df['Ticker'] == ticker]['Weight (%)'].sum()
                    print(f"    - {ticker}: 총 가중치 {original_weight:.2f}%로 합산")
            
            # 3. 가중치 정규화 (100%로 맞추기)
            total_weight = df['Weight (%)'].sum()
            if abs(total_weight - 100.0) > 0.01:
                print(f"  📊 가중치 정규화: {total_weight:.2f}% → 100.00%")
                df['Weight (%)'] = df['Weight (%)'] / total_weight * 100.0
                fixes_for_file.append(f"가중치 정규화: {total_weight:.2f}% → 100.00%")
            
            # 4. 변경사항이 있으면 파일 저장
            if fixes_for_file:
                # 백업 파일 생성
                backup_file = file.replace('.csv', '_backup.csv')
                shutil.copy2(file, backup_file)
                
                # 수정된 파일 저장
                df.to_csv(file, index=False)
                
                fixes_applied.append({
                    'file': filename,
                    'fixes': fixes_for_file,
                    'backup': backup_file
                })
                
                print(f"  💾 수정사항 저장 완료 (백업: {os.path.basename(backup_file)})")
            else:
                print("  ✅ 수정할 내용이 없습니다.")
                
        except Exception as e:
            print(f"  ❌ 파일 처리 오류: {e}")
    
    # 수정 요약 출력
    print(f"\\n📋 총 {len(fixes_applied)}개 파일에서 데이터 품질 문제를 수정했습니다:")
    for fix_info in fixes_applied:
        print(f"\\n📁 {fix_info['file']}:")
        for fix in fix_info['fixes']:
            print(f"  - {fix}")
    
    if fixes_applied:
        print("\\n⚠️  원본 파일들은 '_backup.csv' 형태로 백업되었습니다.")
        print("✅ 데이터 품질 수정이 완료되었습니다. 이제 백테스팅을 다시 실행할 수 있습니다.")
    
    return fixes_applied

# 데이터 품질 자동 수정 실행
fixes = fix_altman_data_quality()


In [ ]:
# 수정 후 데이터 품질 재검사
print("\\n🔍 데이터 품질 수정 후 재검사")
print("=" * 50)
issues_after = check_altman_data_quality()


In [ ]:
# 개선된 데이터로 간단한 백테스팅 비교 실행
print("\\n🚀 개선된 데이터로 Edward Altman 에이전트 백테스팅 비교")
print("=" * 70)

def simple_altman_backtest_comparison():
    """개선 전후 Altman 에이전트 데이터 비교"""
    
    # 1. 개선 전 데이터 (백업 파일들) 분석
    print("📊 개선 전후 데이터 비교 분석")
    print("-" * 50)
    
    portfolio_dir = "results/altman_agent"
    portfolio_files = glob.glob(f"{portfolio_dir}/altman_portfolio_*.csv")
    backup_files = glob.glob(f"{portfolio_dir}/altman_portfolio_*_backup.csv")
    
    comparison_results = []
    
    for file in sorted(portfolio_files):
        if '_backup' in file:
            continue
            
        filename = os.path.basename(file)
        backup_file = file.replace('.csv', '_backup.csv')
        
        if os.path.exists(backup_file):
            print(f"\\n📁 {filename} 비교:")
            
            # 원본 데이터 (백업)
            df_original = pd.read_csv(backup_file)
            # 수정된 데이터
            df_fixed = pd.read_csv(file)
            
            # 비교 결과
            result = {
                'file': filename,
                'original_tickers': len(df_original),
                'fixed_tickers': len(df_fixed),
                'original_weight_sum': df_original['Weight (%)'].sum(),
                'fixed_weight_sum': df_fixed['Weight (%)'].sum(),
                'original_duplicates': len(df_original[df_original.duplicated('Ticker', keep=False)]),
                'fixed_duplicates': len(df_fixed[df_fixed.duplicated('Ticker', keep=False)])
            }
            
            print(f"  📈 티커 수: {result['original_tickers']} → {result['fixed_tickers']}")
            print(f"  📊 가중치 합계: {result['original_weight_sum']:.2f}% → {result['fixed_weight_sum']:.2f}%")
            print(f"  🔄 중복 티커: {result['original_duplicates']}개 → {result['fixed_duplicates']}개")
            
            # 개선 효과 계산
            weight_improvement = abs(100.0 - result['fixed_weight_sum']) < abs(100.0 - result['original_weight_sum'])
            duplicate_improvement = result['fixed_duplicates'] < result['original_duplicates']
            
            if weight_improvement or duplicate_improvement:
                print("  ✅ 데이터 품질 개선됨")
            else:
                print("  ℹ️  변경사항 없음")
                
            comparison_results.append(result)
        else:
            print(f"\\n📁 {filename}: 백업 파일 없음 (원래 깨끗한 데이터)")
    
    # 전체 개선 요약
    if comparison_results:
        print(f"\\n📋 전체 개선 요약:")
        print(f"  - 처리된 파일: {len(comparison_results)}개")
        
        total_duplicate_reduction = sum(r['original_duplicates'] - r['fixed_duplicates'] for r in comparison_results)
        files_with_weight_fix = sum(1 for r in comparison_results if abs(100.0 - r['fixed_weight_sum']) < abs(100.0 - r['original_weight_sum']))
        
        print(f"  - 중복 티커 제거: 총 {total_duplicate_reduction}개")
        print(f"  - 가중치 정규화: {files_with_weight_fix}개 파일")
        
        print("\\n✅ Edward Altman 에이전트의 데이터 품질이 개선되었습니다!")
        print("   이제 백테스팅에서 리밸런싱 시점의 급격한 하락 문제가 해결될 것으로 예상됩니다.")
    else:
        print("\\n📋 개선된 파일이 없습니다. 원본 데이터가 이미 깨끗했을 수 있습니다.")
    
    return comparison_results

# 비교 분석 실행
comparison_results = simple_altman_backtest_comparison()


In [ ]:
# 최종 권장사항 및 다음 단계
print("\\n🎯 최종 권장사항 및 다음 단계")
print("=" * 60)

print("""
📋 데이터 품질 개선 완료 요약:

✅ 해결된 문제들:
  1. 중복 티커 (예: TXN이 2번 등장) → 가중치 합산으로 해결
  2. 티커 오타 (예: CPTR → CPRT) → 자동 수정
  3. 가중치 합계 불일치 (≠ 100%) → 정규화로 해결
  4. 원본 데이터 보존 → 백업 파일 생성

🚀 다음 단계:
  1. 기존 04_multi_agent_backtesting.ipynb를 다시 실행
  2. Edward Altman 에이전트의 성과 곡선 확인
  3. 리밸런싱 시점의 급격한 하락 문제 해결 여부 검증
  4. 다른 에이전트들과의 성과 비교

💡 백테스팅 재실행 방법:
  - 04_multi_agent_backtesting.ipynb를 열고
  - backtester = MultiAgentBacktester() 부터 다시 실행
  - plot_multi_agent_performance()로 결과 확인

⚠️  주의사항:
  - 백업 파일들(_backup.csv)은 삭제하지 마세요
  - 문제가 생기면 백업에서 복원 가능합니다
  - 다른 에이전트들도 같은 방식으로 데이터 품질 검사 권장

🔍 Edward Altman 에이전트 특이사항:
  - Z-score 기반 포트폴리오로 동일 가중치(3%) 사용
  - 중복 티커 문제가 가중치 계산을 왜곡시켰을 가능성
  - 이제 올바른 가중치로 백테스팅 가능
""")

print("✅ 데이터 품질 개선 작업이 완료되었습니다!")
print("   이제 04_multi_agent_backtesting.ipynb에서 개선된 결과를 확인해보세요.")
